In [1]:
!pip install tensorflow tensorflow-model-optimization

In [2]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import os

print("TF version:", tf.__version__)
print("TF-MOT version:", tfmot.__version__)

TF version: 2.19.0
TF-MOT version: 0.8.0


In [8]:
from google.colab import drive
from tensorflow.keras.models import load_model
import tensorflow as tf
drive.mount('/content/drive')
drive_base_path = '/content/drive/MyDrive/spacecraft_anomaly_project/models'

# ===== Paths =====
BASE_DIR = "/content/drive/MyDrive/spacecraft_anomaly_project"  # change if needed
MODELS_DIR = os.path.join(BASE_DIR, "models")


# ===== File names =====
AUTOENCODER_FILE = os.path.join(MODELS_DIR, "autoencoder/lstm_autoencoder.h5")
TTI_MODEL_FILE = os.path.join(MODELS_DIR, "tti_lstm_regressor.keras")

print("📂 Loading models...")
autoencoder = load_model(AUTOENCODER_FILE)
tti_model = load_model(TTI_MODEL_FILE)
print("✅ All models loaded successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Loading models...
✅ All models loaded successfully.


In [11]:
def strip_and_save(model, save_path):
    stripped = tfmot.sparsity.keras.strip_pruning(model)
    stripped.save(save_path)
    print(f"Pruned model saved at: {save_path}")

strip_and_save(autoencoder, "/content/pruned_lstm_autoencoder.h5")
strip_and_save(tti_model, "/content/pruned_tti_regressor.h5")

ValueError: ('Expected model to be a `keras.Model` instance but got: ', <Functional name=functional_1, built=True>)

In [ ]:
def quantize_model_to_tflite(model, save_path):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Post-training quantization
    tflite_model = converter.convert()
    with open(save_path, "wb") as f:
        f.write(tflite_model)
    print(f"TFLite quantized model saved at: {save_path}")

In [ ]:
quantize_model_to_tflite(
    tf.keras.models.load_model("/content/pruned_lstm_autoencoder.h5"),
    "/content/lstm_autoencoder_quant.tflite"
)

quantize_model_to_tflite(
    tf.keras.models.load_model("/content/pruned_tti_regressor.h5"),
    "/content/tti_regressor_quant.tflite"
)

In [ ]:
from pathlib import Path

files = [
    "lstm_autoencoder.h5",
    "tti_regressor.h5",
    "pruned_lstm_autoencoder.h5",
    "pruned_tti_regressor.h5",
    "lstm_autoencoder_quant.tflite",
    "tti_regressor_quant.tflite"
]

for file in files:
    file_path = Path(f"/content/{file}")
    if file_path.exists():
        size_kb = file_path.stat().st_size / 1024
        print(f"{file}: {size_kb:.2f} KB")